In [15]:
# system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
import scipy
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns

import pickle


#import seaborn

In [16]:
#style settings
sns.set(style='whitegrid', rc={'axes.facecolor': 'white'})

#sns.set_style({'font.family':'Arial', 'font.serif':'Times New Roman'})

In [17]:
def read_manipulate_psychopy(psychopy_path):
    
    psychopy_df = pd.read_csv(psychopy_path)
    col_list = [col for col in psychopy_df.columns if col.endswith('_ts')]
    col_list.insert(0, "Reference_time")
    psychopy_df_selected = psychopy_df[col_list]
    
    return psychopy_df_selected


def get_nonNan_list_psychopy(psychopy_df, col_name):
    
    
    selected_vals=[val for val in psychopy_df[col_name].to_list() if not(math.isnan(val))]
    
    return selected_vals

In [18]:
def get_offset_timestamp(timestamp, offset_mins):
    
    time_zone = 'Europe/Berlin'
    tz = pytz.timezone(time_zone)
    local_time = dt.datetime.fromtimestamp(timestamp, tz)
    time_change = dt.timedelta(minutes=offset_mins)
    new_time = local_time + time_change
    new_timestamp =  dt.datetime.timestamp(new_time)
    return new_timestamp


def get_time_diff_saliva_ts(timestamp_1, timestamp_2):
    
    time_zone = 'Europe/Berlin'
    tz = pytz.timezone(time_zone)
    local_time_1 = dt.datetime.fromtimestamp(timestamp_1, tz)
    local_time_2 = dt.datetime.fromtimestamp(timestamp_2, tz)
    difference = local_time_2 - local_time_1
    diff_seconds = difference.total_seconds() 
    diff_minutes =diff_seconds/60
    
    return diff_minutes

In [19]:
data_dir_path = "/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/data_sony_digiRelax/wp3"
#data_dir_path = "D:/Datasets/data_sony_digiRelax/study"
sub_dirs=next(os.walk(data_dir_path))[1]
sub_dirs.sort(reverse=False)

dest_path = "/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/data_sony_digiRelax/result_psychopy_extracted_info/saliva_time_mins"

In [20]:

list_saliva_probe_ts= ["saliva_probe_1_ts", "saliva_probe_2_ts","saliva_probe_3_ts", "saliva_probe_4_ts"]
extracted_info_dict = {}
all_files_list=[]
extracted_info_dict["file_name"]=[]

for sub_dir in sub_dirs:
    
    sub_dir_path= os.path.join(data_dir_path, sub_dir)
    psychopy_file_path = os.path.join(sub_dir_path, "Psychopy_data")
    psychopy_file_path = glob.glob(psychopy_file_path+ "/*.csv")[0]
    
    dir_path, file_name=os.path.split(psychopy_file_path)
    extracted_info_dict["file_name"].append(file_name)
    sub_id=file_name[:-4]
    print(sub_id)
    
    
    psychopy_df = read_manipulate_psychopy(psychopy_file_path)
    
    ind_file_list = []
    for ind, sel_saliva in enumerate(list_saliva_probe_ts):
        
    
        if ind==0:

            prev_saliva=sel_saliva
            prev_saliva_ts = get_nonNan_list_psychopy(psychopy_df, prev_saliva)[0]

        else:
            curr_saliva = sel_saliva
            curr_saliva_ts = get_nonNan_list_psychopy(psychopy_df, curr_saliva)[0]
            time_curr_minus_prev=get_time_diff_saliva_ts(prev_saliva_ts, curr_saliva_ts)
            ind_file_list.append(time_curr_minus_prev)

            print("Time between:  " + curr_saliva +  " - " + prev_saliva + " = " + str(time_curr_minus_prev) + " mins")
            prev_saliva =curr_saliva
            prev_saliva_ts = curr_saliva_ts
            
            
        
        
    all_files_list.append(ind_file_list)
            
    #print("-----------------------------------------------------------------")
    
    
extracted_info_dict["saliva_probe_2_1"] = []
extracted_info_dict["saliva_probe_3_2"] = []
extracted_info_dict["saliva_probe_4_3"] = []

for sel_file in all_files_list:
    
    #print(sel_file)
    extracted_info_dict["saliva_probe_2_1"].append(sel_file[0])
    extracted_info_dict["saliva_probe_3_2"].append(sel_file[1])
    extracted_info_dict["saliva_probe_4_3"].append(sel_file[2])


    
df_export = pd.DataFrame.from_dict(extracted_info_dict)

dest_path_saliva_time = os.path.join(dest_path,"saliva_time_interval_mins.csv")

df_export.to_csv(dest_path_saliva_time)
    

VP004_081123_DigiRelax_Experiment_2023-11-08_10h28.36.317
Time between:  saliva_probe_2_ts - saliva_probe_1_ts = 20.181002133333333 mins
Time between:  saliva_probe_3_ts - saliva_probe_2_ts = 24.436710816666668 mins
Time between:  saliva_probe_4_ts - saliva_probe_3_ts = 15.819049966666666 mins
VP005_081123_DigiRelax_Experiment_2023-11-08_14h17.35.226
Time between:  saliva_probe_2_ts - saliva_probe_1_ts = 20.065377716666667 mins
Time between:  saliva_probe_3_ts - saliva_probe_2_ts = 24.588638166666666 mins
Time between:  saliva_probe_4_ts - saliva_probe_3_ts = 13.110345966666667 mins
VP006_091123_DigiRelax_Experiment_2023-11-09_10h17.40.415
Time between:  saliva_probe_2_ts - saliva_probe_1_ts = 21.783582466666665 mins
Time between:  saliva_probe_3_ts - saliva_probe_2_ts = 25.96337511666667 mins
Time between:  saliva_probe_4_ts - saliva_probe_3_ts = 11.944784083333333 mins
VP007_091123_DigiRelax_Experiment_2023-11-09_14h19.56.411
Time between:  saliva_probe_2_ts - saliva_probe_1_ts = 21.

In [21]:
len(df_export["file_name"].values)

82

In [22]:
saliva_ts_path = "/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/data_sony_digiRelax/result_psychopy_extracted_info/saliva_time_mins/saliva_time_interval_mins.csv"

In [23]:
saliva_ts_df = pd.read_csv(saliva_ts_path)

In [24]:
saliva_ts_df
    
    

,Unnamed: 0,file_name,saliva_probe_2_1,saliva_probe_3_2,saliva_probe_4_3
0,0,VP004_081123_DigiRelax_Experiment_2023-11-08_1...,20.181002,24.436711,15.819050
1,1,VP005_081123_DigiRelax_Experiment_2023-11-08_1...,20.065378,24.588638,13.110346
2,2,VP006_091123_DigiRelax_Experiment_2023-11-09_1...,21.783582,25.963375,11.944784
3,3,VP007_091123_DigiRelax_Experiment_2023-11-09_1...,21.156972,25.364625,11.681857
4,4,VP008_101123_DigiRelax_Experiment_2023-11-10_1...,20.508294,24.699030,14.529951
...,...,...,...,...,...
77,77,VP082_260124_DigiRelax_Experiment_2024-01-26_1...,20.091112,24.996844,12.610933
78,78,VP083_310124_DigiRelax_Experiment_2024-01-31_0...,20.208016,24.389188,14.174565
79,79,VP084_310124_DigiRelax_Experiment_2024-01-31_1...,19.519660,24.223101,11.775196
80,80,VP085_010224_DigiRelax_Experiment_2024-02-01_0...,19.886880,24.829150,11.644762


array(['VP004_081123_DigiRelax_Experiment_2023-11-08_10h28.36.317.csv',
       'VP005_081123_DigiRelax_Experiment_2023-11-08_14h17.35.226.csv',
       'VP006_091123_DigiRelax_Experiment_2023-11-09_10h17.40.415.csv',
       'VP007_091123_DigiRelax_Experiment_2023-11-09_14h19.56.411.csv',
       'VP008_101123_DigiRelax_Experiment_2023-11-10_10h24.14.323.csv',
       'VP009_101123_DigiRelax_Experiment_2023-11-10_14h16.47.223.csv',
       'VP010_131123_DigiRelax_Experiment_2023-11-13_09h54.08.913.csv',
       'VP011_131123_DigiRelax_Experiment_2023-11-13_14h42.51.249.csv',
       'VP012_141123_DigiRelax_Experiment_2023-11-14_09h47.48.508.csv',
       'VP013_151123_DigiRelax_Experiment_2023-11-15_09h52.50.734.csv',
       'VP014_151123_DigiRelax_Experiment_2023-11-15_14h23.06.117.csv',
       'VP015_161123_DigiRelax_Experiment_2023-11-16_09h47.53.124.csv',
       'VP016_161123_DigiRelax_Experiment_2023-11-16_14h46.57.355.csv',
       'VP017_171123_DigiRelax_Experiment_2023-11-17_14h43.13.41